In [1]:
# In this version of OGA with random dictionaries, we use QMC to evaluate the loss function. 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_dtype(torch.float64)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1


def generate_relu_dict4plusD_QMC(dim, s,N0):
    # s*N0 is the total number of samples 
    samples = torch.rand(s*N0,dim)  

    # for i in range(s-1):
        # samples = torch.cat([samples,Sob.draw(N0).double()],0)
    # Form the transformation matrix and shift vector 
    diagonal = torch.ones(dim)*pi  
    diagonal[-1] =  2*dim**0.5
    diagonal[-2] = 2*pi 
    T = torch.diag(diagonal)

    shift = torch.zeros(dim)
    shift[-1] = -dim**0.5 
    samples = samples@T + shift 

    Wb_tensor = torch.ones(s*N0,dim+1) # each neuron parameter stored in rows  
    for i in range(dim): # 0, 1, ... dim-1 
        for j in range(i+1): # 0, 1, ... i 
            if i == 0: 
                Wb_tensor[:,i] = Wb_tensor[:,i]*torch.cos(samples[:,j])
            if i == (dim - 1):
                if j != i:
                    Wb_tensor[:,i] = Wb_tensor[:,i] * torch.sin(samples[:,j]) 
            if i != 0 and i != (dim - 1): 
                if j != i: 
                    Wb_tensor[:,i] = Wb_tensor[:,i] * torch.sin(samples[:,j]) 
                else: 
                    Wb_tensor[:,i] = Wb_tensor[:,i] * torch.cos(samples[:,j]) 
            
    Wb_tensor[:,dim] = samples[:,-1] 

    return Wb_tensor.to(device)


def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights.to(device), integration_points.to(device) 


def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct"):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    
    # new batched operation 
    n = b.size(0)
    M = integration_points.size(0)
    
    total_size = n * M # memory, number of floating numbers 
    num_batch = total_size//(2**30) + 1 # divide according to memory
    batch_size = M//num_batch
    start_ind = 0
    end_ind = 0 
    jac = torch.zeros(b.size(0),b.size(0)).to(device)
    rhs = torch.zeros(b.size(0),1).to(device)
#     print("mat assemble, number batches: ",num_batch)
    for j in range(0,M,batch_size): 
        end_ind = j + batch_size
        basis_value_col = F.relu(integration_points[j:end_ind] @ w.t()+ b)**(model.k) 
        weighted_basis_value_col = basis_value_col * weights[j:end_ind] 
        jac += weighted_basis_value_col.t() @ basis_value_col 
        rhs += weighted_basis_value_col.t() @ (target(integration_points[j:end_ind,:])) 
        
    print("jac: ", jac.device)
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 

def OGAL2FittingReLU4Dplus_QMC(my_model,target,s,N0,num_epochs, M, k =1, linear_solver = "direct", num_batches = 1): 
    
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    # samples for QMC integral
    dim = 10 
    start_time = time.time()
    # Sob_integral = torch.quasirandom.SobolEngine(dimension =4, scramble= False, seed=None) 
    # integration_points = Sob_integral.draw(M).double() 
    # integration_points = integration_points.to(device)
    weights, integration_points = MonteCarlo_Sobol_dDim_weights_points(M ,d = dim) 
    _, integration_points_test = MonteCarlo_Sobol_dDim_weights_points(M*2 ,d = dim) 
    print("generate sob sequence:", time.time() - start_time) 

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.mean(func_values_sqrd)**0.5
    all_start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict4plusD_QMC(dim, s,N0).t()   
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            

            total_size = num_neuron * M 
            num_batch = total_size//(2**30) + 1 # divide according to memory
            batch_size = M//num_batch
            end_ind = 0 

            func_values = torch.zeros(M,1).to(device)

            for j in range(0,M,batch_size): 
                end_ind = j + batch_size
                func_values[j:end_ind,:] = target(integration_points[j:end_ind,:]) - my_model(integration_points[j:end_ind,:]).detach()

        start_time = time.time() 
        
        M = integration_points.size(0)
        N = s*N0 
        output = torch.zeros(N,1)
        num_batches = (N*M)//2**30 + 1 # decide num_batches according to memory 
        batch_size = N//num_batches 
        print("argmax batch num, ", num_batches)
        for j in range(0,N,batch_size): 

            end_index = j + batch_size  
            basis_values_batch = (F.relu( torch.matmul(integration_points,relu_dict_parameters[0:dim, j:end_index] ) - relu_dict_parameters[dim, j:end_index])**k).T # uses broadcasting    
            output[j:end_index,0]  = (torch.abs(torch.matmul(basis_values_batch,func_values))/M)[:,0]
            
        neuron_index = torch.argmax(output.flatten())
        
#         basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T # uses broadcasting
#         output = torch.abs(torch.matmul(basis_values,func_values))/M # 
#         neuron_index = torch.argmax(output.flatten())
        print("argmax time taken, ", time.time() - start_time)
        
        list_w.append(relu_dict_parameters[0:dim, neuron_index]) # 
        list_b.append(-relu_dict_parameters[dim,neuron_index])
        num_neuron += 1
        my_model = model(dim,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        start_time = time.time() 
        sol = minimize_linear_layer_explicit_assemble(my_model,target,weights,integration_points, solver)
        print("\t\t time taken minimize linear layer: ",time.time() - start_time) 
        my_model.fc2.weight.data[0,:] = sol[:]

        # calculate the test error
#         func_values = target(integration_points_test) - my_model(integration_points_test).detach()
        
        M2 = integration_points_test.size(0)
        total_size = num_neuron * M2 
        num_batch = total_size//(2**30) + 1 # divide according to memory
        batch_size = M//num_batch
        end_ind = 0 

        func_values_sqrd = torch.zeros(M2,1)  
        
        for j in range(0,M2,batch_size): 
            end_ind = j + batch_size
            func_values_sqrd[j:end_ind,:] = target(integration_points_test[j:end_ind,:]) - my_model(integration_points_test[j:end_ind,:]).detach()
        
        func_values_sqrd = func_values_sqrd**2 

        #Todo Done 
        err[i+1]= torch.mean(func_values_sqrd)**0.5
        print("current error: ",err[i+1]) 
    print("total duration: ",time.time() - all_start_time)
    return err, my_model

def print_convergence_order(err, neuron_num_exponent): 

    neuron_nums = [2**j for j in range(2,neuron_num_exponent)]
    err_list = [err[i] for i in neuron_nums ] 

    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))



## Random dictionary 

Gaussian function 

In [ ]:
if __name__ == "__main__": 

    def target(x): ## Gaussian function in dimension 10  
        d = 10 
        cn =   7.03/d 
        return torch.exp(-torch.sum( cn**2 * (x - 0.5)**2,dim = 1, keepdim = True)) 
    
    save = True 
    experiment_label = "ex1"
    for k in [4]: 
        s = 1 
        for N0 in [2**16]: 
            print()
            print() 
            exponent = 10    
            num_epochs=  2**exponent 
            M = 2**20 # around2**19 50w 
            print(M)
            my_model = None 
            
            err, my_model = OGAL2FittingReLU4Dplus_QMC(my_model,target, \
                        s,N0,num_epochs, M, k = k, linear_solver = "direct", num_batches = 4)
            
            if save: 
                filename = experiment_label + "_err_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(k,s * N0,num_epochs)
                torch.save(err,filename)
                filename = experiment_label + "_model_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(k,s * N0,num_epochs)
                torch.save(my_model.state_dict(),filename) 

            print_convergence_order(err,exponent+1) 




1048576
generate sob sequence: 0.8223865032196045
using linear solver:  direct
epoch:  1	argmax batch num,  17
argmax time taken,  1.0435035228729248
jac:  cuda:0
assembling the matrix time taken:  0.0006909370422363281
solving Ax = b time taken:  0.08925962448120117
		 time taken minimize linear layer:  0.0900111198425293
current error:  tensor(0.0902)
epoch:  2	argmax batch num,  17
argmax time taken,  0.6929843425750732
jac:  cuda:0
assembling the matrix time taken:  0.0008399486541748047
solving Ax = b time taken:  0.00043702125549316406
		 time taken minimize linear layer:  0.0013053417205810547
current error:  tensor(0.0786)
epoch:  3	argmax batch num,  17
argmax time taken,  0.6936492919921875
jac:  cuda:0
assembling the matrix time taken:  0.00046896934509277344
solving Ax = b time taken:  0.0007867813110351562
		 time taken minimize linear layer:  0.0012819766998291016
current error:  tensor(0.0772)
epoch:  4	argmax batch num,  17
argmax time taken,  0.6931004524230957
jac: 

argmax time taken,  0.6940367221832275
jac:  cuda:0
assembling the matrix time taken:  0.0005385875701904297
solving Ax = b time taken:  0.0023245811462402344
		 time taken minimize linear layer:  0.00289154052734375
current error:  tensor(0.0390)
epoch:  31	argmax batch num,  17
argmax time taken,  0.6938164234161377
jac:  cuda:0
assembling the matrix time taken:  0.0006146430969238281
solving Ax = b time taken:  0.002302408218383789
		 time taken minimize linear layer:  0.0029604434967041016
current error:  tensor(0.0380)
epoch:  32	argmax batch num,  17
argmax time taken,  0.6938595771789551
jac:  cuda:0
assembling the matrix time taken:  0.0005586147308349609
solving Ax = b time taken:  0.002359628677368164
		 time taken minimize linear layer:  0.002962827682495117
current error:  tensor(0.0369)
epoch:  33	argmax batch num,  17
argmax time taken,  0.6939423084259033
jac:  cuda:0
assembling the matrix time taken:  0.0007011890411376953
solving Ax = b time taken:  0.00266003608703613

argmax time taken,  0.6979649066925049
jac:  cuda:0
assembling the matrix time taken:  0.00044608116149902344
solving Ax = b time taken:  0.004204273223876953
		 time taken minimize linear layer:  0.0046923160552978516
current error:  tensor(0.0190)
epoch:  61	argmax batch num,  17
argmax time taken,  0.6979336738586426
jac:  cuda:0
assembling the matrix time taken:  0.0004737377166748047
solving Ax = b time taken:  0.004271745681762695
		 time taken minimize linear layer:  0.004786252975463867
current error:  tensor(0.0188)
epoch:  62	argmax batch num,  17
argmax time taken,  0.6980843544006348
jac:  cuda:0
assembling the matrix time taken:  0.0005042552947998047
solving Ax = b time taken:  0.004353523254394531
		 time taken minimize linear layer:  0.004921913146972656
current error:  tensor(0.0186)
epoch:  63	argmax batch num,  17
argmax time taken,  0.698331356048584
jac:  cuda:0
assembling the matrix time taken:  0.0006115436553955078
solving Ax = b time taken:  0.00431728363037109

argmax time taken,  0.6988556385040283
jac:  cuda:0
assembling the matrix time taken:  0.0005512237548828125
solving Ax = b time taken:  0.0077953338623046875
		 time taken minimize linear layer:  0.00838160514831543
current error:  tensor(0.0141)
epoch:  91	argmax batch num,  17
argmax time taken,  0.7020432949066162
jac:  cuda:0
assembling the matrix time taken:  0.0005867481231689453
solving Ax = b time taken:  0.007503509521484375
		 time taken minimize linear layer:  0.008138895034790039
current error:  tensor(0.0140)
epoch:  92	argmax batch num,  17
argmax time taken,  0.698824405670166
jac:  cuda:0
assembling the matrix time taken:  0.0005199909210205078
solving Ax = b time taken:  0.007448673248291016
		 time taken minimize linear layer:  0.00801396369934082
current error:  tensor(0.0139)
epoch:  93	argmax batch num,  17
argmax time taken,  0.7023210525512695
jac:  cuda:0
assembling the matrix time taken:  0.0005252361297607422
solving Ax = b time taken:  0.007575035095214844
	

argmax time taken,  0.703338623046875
jac:  cuda:0
assembling the matrix time taken:  0.0005948543548583984
solving Ax = b time taken:  0.008810281753540039
		 time taken minimize linear layer:  0.009466171264648438
current error:  tensor(0.0111)
epoch:  121	argmax batch num,  17
argmax time taken,  0.703315258026123
jac:  cuda:0
assembling the matrix time taken:  0.0004620552062988281
solving Ax = b time taken:  0.009105920791625977
		 time taken minimize linear layer:  0.009614706039428711
current error:  tensor(0.0110)
epoch:  122	argmax batch num,  17
argmax time taken,  0.7029469013214111
jac:  cuda:0
assembling the matrix time taken:  0.0005004405975341797
solving Ax = b time taken:  0.0095977783203125
		 time taken minimize linear layer:  0.010160207748413086
current error:  tensor(0.0110)
epoch:  123	argmax batch num,  17
argmax time taken,  0.7039222717285156
jac:  cuda:0
assembling the matrix time taken:  0.0005135536193847656
solving Ax = b time taken:  0.009187459945678711


...

argmax time taken,  3.3960556983947754
jac:  cuda:0
assembling the matrix time taken:  0.0004742145538330078
solving Ax = b time taken:  0.029659271240234375
		 time taken minimize linear layer:  0.030159950256347656
current error:  tensor(0.0063)
epoch:  268	argmax batch num,  65
argmax time taken,  3.3951990604400635
jac:  cuda:0
assembling the matrix time taken:  0.0004868507385253906
solving Ax = b time taken:  0.02880072593688965
		 time taken minimize linear layer:  0.02931380271911621
current error:  tensor(0.0063)
epoch:  269	argmax batch num,  65
argmax time taken,  3.3960909843444824
jac:  cuda:0
assembling the matrix time taken:  0.0004737377166748047
solving Ax = b time taken:  0.029802560806274414
		 time taken minimize linear layer:  0.030302762985229492
current error:  tensor(0.0063)
epoch:  270	argmax batch num,  65
argmax time taken,  3.3951761722564697
jac:  cuda:0
assembling the matrix time taken:  0.0004863739013671875
solving Ax = b time taken:  0.03413939476013183

argmax time taken,  3.3961708545684814
jac:  cuda:0
assembling the matrix time taken:  0.00048279762268066406
solving Ax = b time taken:  0.0314943790435791
		 time taken minimize linear layer:  0.03200340270996094
current error:  tensor(0.0061)
epoch:  298	argmax batch num,  65
argmax time taken,  3.3967621326446533
jac:  cuda:0
assembling the matrix time taken:  0.0004665851593017578
solving Ax = b time taken:  0.0315241813659668
		 time taken minimize linear layer:  0.032016754150390625
current error:  tensor(0.0061)
epoch:  299	argmax batch num,  65
argmax time taken,  3.3966143131256104
jac:  cuda:0
assembling the matrix time taken:  0.00048089027404785156
solving Ax = b time taken:  0.03165316581726074
		 time taken minimize linear layer:  0.032160282135009766
current error:  tensor(0.0061)
epoch:  300	argmax batch num,  65
argmax time taken,  3.3965542316436768
jac:  cuda:0
assembling the matrix time taken:  0.0004839897155761719
solving Ax = b time taken:  0.0315861701965332
		

argmax time taken,  3.3924763202667236
jac:  cuda:0
assembling the matrix time taken:  0.0005030632019042969
solving Ax = b time taken:  0.04141736030578613
		 time taken minimize linear layer:  0.041947126388549805
current error:  tensor(0.0056)
epoch:  328	argmax batch num,  65
argmax time taken,  3.392421007156372
jac:  cuda:0
assembling the matrix time taken:  0.0005252361297607422
solving Ax = b time taken:  0.037322282791137695
		 time taken minimize linear layer:  0.03787374496459961
current error:  tensor(0.0056)
epoch:  329	argmax batch num,  65
argmax time taken,  3.3924753665924072
jac:  cuda:0
assembling the matrix time taken:  0.0005054473876953125
solving Ax = b time taken:  0.04151439666748047
		 time taken minimize linear layer:  0.042046546936035156
current error:  tensor(0.0056)
epoch:  330	argmax batch num,  65
argmax time taken,  3.3921573162078857
jac:  cuda:0
assembling the matrix time taken:  0.0005314350128173828
solving Ax = b time taken:  0.049269676208496094


argmax time taken,  3.3933656215667725
jac:  cuda:0
assembling the matrix time taken:  0.0005202293395996094
solving Ax = b time taken:  0.04340052604675293
		 time taken minimize linear layer:  0.04394698143005371
current error:  tensor(0.0050)
epoch:  358	argmax batch num,  65
argmax time taken,  3.3934664726257324
jac:  cuda:0
assembling the matrix time taken:  0.0005242824554443359
solving Ax = b time taken:  0.043234825134277344
		 time taken minimize linear layer:  0.04378557205200195
current error:  tensor(0.0049)
epoch:  359	argmax batch num,  65
argmax time taken,  3.392958164215088
jac:  cuda:0
assembling the matrix time taken:  0.0005235671997070312
solving Ax = b time taken:  0.04345059394836426
		 time taken minimize linear layer:  0.04400014877319336
current error:  tensor(0.0049)
epoch:  360	argmax batch num,  65
argmax time taken,  3.3937065601348877
jac:  cuda:0
assembling the matrix time taken:  0.0005214214324951172
solving Ax = b time taken:  0.043382883071899414
		

argmax time taken,  3.399839401245117
jac:  cuda:0
assembling the matrix time taken:  0.0005393028259277344
solving Ax = b time taken:  0.048120737075805664
		 time taken minimize linear layer:  0.04868650436401367
current error:  tensor(0.0043)
epoch:  388	argmax batch num,  65


In [10]:
def compute_rate(k,d):
    return 0.5 + (2*k +1)/(2*d)

for k in range(1,6): 
    print(k, compute_rate(k,10))

1 0.65
2 0.75
3 0.85
4 0.95
5 1.05


In [33]:
dim = 10 
points = torch.rand(12,dim).to(device)
approx_values = my_model(points)
print(approx_values)


print("ture values")
print(target(points))

tensor([[0.7069],
        [0.5646],
        [0.5421],
        [0.6234],
        [0.7319],
        [0.6844],
        [0.5897],
        [0.7394],
        [0.5824],
        [0.5579],
        [0.6385],
        [0.7090]], device='cuda:0', grad_fn=<MmBackward0>)
ture values
tensor([[0.7066],
        [0.5647],
        [0.5421],
        [0.6238],
        [0.7315],
        [0.6843],
        [0.5901],
        [0.7392],
        [0.5826],
        [0.5581],
        [0.6386],
        [0.7088]], device='cuda:0')


In [ ]:
k = 1, N0 = 2**7 
4		tensor(0.0749)		*
8		tensor(0.0689)		tensor(0.1203)
16		tensor(0.0619)		tensor(0.1563)
32		tensor(0.0547)		tensor(0.1775)
64		tensor(0.0445)		tensor(0.2971)
128		tensor(0.0367)		tensor(0.2776)
256		tensor(0.0261)		tensor(0.4910)
512		tensor(0.0182)		tensor(0.5241) 

k = 1, N0 = 2**9, # test_points = M 
4		tensor(0.0750)		*
8		tensor(0.0701)		tensor(0.0981)
16		tensor(0.0599)		tensor(0.2269)
32		tensor(0.0500)		tensor(0.2615)
64		tensor(0.0402)		tensor(0.3141)
128		tensor(0.0279)		tensor(0.5245)
256		tensor(0.0168)		tensor(0.7364)
512		tensor(0.0100)		tensor(0.7507)

k = 1, N0 = 2**9, # test_points = 2 * M 
4		tensor(0.0749)		*
8		tensor(0.0682)		tensor(0.1355)
16		tensor(0.0567)		tensor(0.2666)
32		tensor(0.0491)		tensor(0.2071)
64		tensor(0.0397)		tensor(0.3076)
128		tensor(0.0258)		tensor(0.6189)
256		tensor(0.0153)		tensor(0.7537)
512		tensor(0.0094)		tensor(0.7103)


k = 4, N0 =2**10, test_points = 2*M 
4		tensor(0.1333)		*
8		tensor(0.0888)		tensor(0.5859)
16		tensor(0.0810)		tensor(0.1327)
32		tensor(0.0504)		tensor(0.6835)
64		tensor(0.0204)		tensor(1.3081)
128		tensor(0.0087)		tensor(1.2292)
256		tensor(0.0064)		tensor(0.4499)
512		tensor(0.0034)		tensor(0.9121)
1024		tensor(0.0004)		tensor(3.0958)